## Постановка задачи
У нас появился запрос из отдела продаж и маркетинга. Как вы знаете «МегаФон» предлагает обширный набор различных услуг своим абонентам. При этом разным пользователям интересны разные услуги. Поэтому необходимо построить  алгоритм, который для каждой пары пользователь-услуга определит вероятность подключения услуги.
Данные
В качестве исходных данных вам будет доступна информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.
Отдельным набором данных будет являться нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.
#### Данные: 
train и test разбиты по периодам – на train доступно 4 месяцев, а на test отложен последующий месяц. 
Итого, в качестве входных данных будут представлены:

●	data_train.csv: id, vas_id, buy_time, target

●	features.csv.zip: id, <feature_list> 

#### Тестовый набор:

●	data_test.csv: id, vas_id, buy_time

target - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно. 
buy_time - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime.
id - идентификатор абонента
vas_id - подключаемая услуга
Примечание: Размер файла features.csv в распакованном виде весит 20 гб, для работы  с ним можно воспользоваться pandas.read_csv, либо можно воспользоваться библиотекой Dask.
Метрика
Скоринг будет осуществляться функцией f1, невзвешенным образом, как например делает функция sklearn.metrics.f1_score(…, average=’macro’).

In [1]:
import pandas as pd
#import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
FEATURES_PATH = 'data/features.csv'
DATA_TRAIN_PATH = 'data/data_train.csv'
DATA_TEST_PATH = 'data/data_test.csv'
DATA_TRAIN_MERGE_PATH = 'data/data_train_merge.csv'
DATA_TEST_MERGE_PATH = 'data/data_test_merge.csv'

##### Загрузим файл с признаками клиента. Файл весит более 20Гб, прочитаем его разбив на куски.

In [2]:
df_features = pd.read_csv(FEATURES_PATH,chunksize = 100000, iterator = True, delimiter='\\t')

##### Загрузим обучающий набор 

In [5]:
df_train = pd.read_csv(DATA_TRAIN_PATH,delimiter=',')

In [6]:
df_train.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0
3,3,3535012,5.0,1535922000,0.0
4,4,1693214,1.0,1535922000,0.0


In [16]:
df_train = df_train.drop(columns = ['Unnamed: 0'])

##### buy_time - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime.
##### target - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно. 

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831653 entries, 0 to 831652
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  831653 non-null  int64  
 1   id          831653 non-null  int64  
 2   vas_id      831653 non-null  float64
 3   buy_time    831653 non-null  int64  
 4   target      831653 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 31.7 MB


##### Отлично! Видим, что DataFrame содержит 831653 записей и нет пропущенных значений. Проверим на уникальность значений.

In [8]:
df_train.nunique()

Unnamed: 0    831653
id            806613
vas_id             8
buy_time          26
target             2
dtype: int64

##### target - целевая переменная имеет бинарный признак, id - идентификатор абонента имеет не уникальные значения, свидетельствует о том, что клиенту было предложено несколько попыток с предложением о подключении услуги. 8 услуг, 26 дат с предложением услуг.

##### Избавимся от дублей если они есть.

In [9]:
df_train = df_train.drop_duplicates()

##### Отфильтруем DataFrame профилей абонентов по id из обучающего набора. Иными словами выберем только те профили клиентов которые присутствуют в обучающем наборе. 

In [10]:
df_features_new = pd.DataFrame() # пустой фрейм
for chunk_iter in df_features: # цикл по кускам dataframe
  filtr_id = chunk_iter['id'].isin(df_train.id) # создаём фильтр по куску  
  chunk = chunk_iter[filtr_id] # фильтруем записи в куске
  df_features_new = pd.concat([df_features_new,chunk]) # склеиваем отфильтрованные куски в dataframe 
  #print(chunk_iter)  
  #print(chunk)

In [12]:
df_features_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 834168 entries, 13 to 4512521
Columns: 255 entries, id to 252
dtypes: float64(253), int64(2)
memory usage: 1.6 GB


#####  dataframe значительно уменьшился, до 1.6 Gb 

In [13]:
df_features_new['buy_time'] = df_features_new.buy_time.apply(datetime.fromtimestamp)

In [14]:
df_train['buy_time'] = df_train.buy_time.apply(datetime.fromtimestamp)

In [15]:
df_train.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,2018-09-17 01:00:00,0.0
1,1,1454121,4.0,2018-07-16 01:00:00,0.0
2,2,2458816,1.0,2018-08-13 01:00:00,0.0
3,3,3535012,5.0,2018-09-03 01:00:00,0.0
4,4,1693214,1.0,2018-09-03 01:00:00,0.0


In [17]:
df_train = df_train.sort_values(by=['buy_time'])

In [18]:
df_features_new = df_features_new.sort_values(by=['buy_time'])

In [19]:
dt_merge = pd.merge_asof(df_train, df_features_new, on='buy_time', by='id')

In [20]:
dt_merge.isna().sum()

id               0
vas_id           0
buy_time         0
target           0
0           422929
             ...  
248         422929
249         422929
250         422929
251         422929
252         422929
Length: 257, dtype: int64

##### При слиянии получилось большое количестово колонок с пустыми значениями. Удалим их.

In [21]:
dt_merge = dt_merge.dropna()

In [22]:
dt_merge.nunique()

id          397259
vas_id           8
buy_time        26
target           2
0            32842
             ...  
248           1583
249            213
250           1398
251            209
252             19
Length: 257, dtype: int64

In [51]:
dt_merge.to_csv(DATA_TRAIN_MERGE_PATH, index = False)

In [52]:
import FileProcessing

In [ ]:
data_test_merge = FileProcessing.preparing(DATA_TEST_PATH, FEATURES_PATH)

In [ ]:
data_test_merge.to_csv(DATA_TRAIN_MERGE_PATH, index = False)